# Stocatistic block model and Graph neural network

V 4.2 adds a new type of GNN

V 4.1 is the version for the condivision with other people

## Download pack

### Pack for colab

In [ ]:
!pip install torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu113.html

## Initialization of the program

### Import libraries

In [ ]:
# Importing module
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.cluster import KMeans
from torch_geometric.nn import GCNConv
from tqdm.notebook import trange
from itertools import permutations

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Functions

In [ ]:
# Show the adiacency matrix
def show(A):
    f = plt.figure()
    f.set_figwidth(7.5)
    f.set_figheight(7.5)
    plt.imshow(A, cmap='Greys', interpolation='nearest')
    plt.axis('off')
    plt.show()

# Generation of a random graph for the SBM problem with 2 communities
def random_graph_generator(n, p, q, k, graph_num, use_MSE=False, shuffle=True):
    np.random.seed(graph_num)

    # Number of nodes per cluster
    n_c = n // k

    # Initialize labels and indices
    labels = np.reshape(np.indices((k, n_c))[0,:,:], n)
    indices = np.arange(n)

    # Initialize Adjacency matrix and the triangular indices
    A = np.zeros((n, n))
    triu_inds = np.triu_indices(n_c, k=1)

    # Fill the upper triangle
    for i in range(k):
        for j in range(i, k):
            if j == i:
                # in cluster
                cluster_i = np.random.choice([0, 1], size=(n_c * (n_c - 1) // 2), p = [1-p, p])
                A[triu_inds[0] + i * n_c, triu_inds[1] + i * n_c] = cluster_i
            else:
                # out of clusters
                between_clusters_i_j = np.random.choice([0, 1], size=(n_c, n_c), p=[1-q, q])
                A[n_c * i : n_c * (i + 1), n_c * j : n_c * (j + 1)] = between_clusters_i_j

    # Fill the under triangle
    A = A + A.T

    if shuffle:
        # Shuffle the adjacency matrix
        np.random.shuffle(indices)
        A = A[indices][:, indices]

        # Shuffle the clusters
        labels = labels[indices]

    # initialize the label list and the permutations
    labels_list = []
    perm = list(permutations(range(k)))

    if use_MSE:
        labels = labels * 2 - 1
        labels_list.append(labels)
        labels_list.append(-labels)
    else:
        new_labels = np.zeros(n).astype(int)

        for i in range(len(perm)):
            # Permute the labels
            for l in range(k):
                new_labels[labels == l] = perm[i][l]

            # initialize the array for the loss
            labels_i = np.zeros((n,k))

            # create the labels for the cross entropy loss
            for j in range(n):
                labels_i[j, new_labels[j]] = 1

            labels_list.append(labels_i)

    # Create the edge index
    edge_index = np.array(np.nonzero(A))

    return A, edge_index, labels_list

# Training
def training_detection(model, optim, criterion, n, p, q, k, train_graph_num, par_lambda, print_loss = True, use_MSE=False, shuffle=False):
    checkpoint_print = int(train_graph_num/10)
    checkpoint_save = int(train_graph_num/100)

    X = torch.from_numpy(np.eye(n)).float().to(device)

    losses = []
    mean_losses = []

    for graph_num in trange(train_graph_num):

        # Initilize a training graph
        A, edge_index, labels_list = random_graph_generator(n, p, q, k, graph_num, use_MSE=use_MSE, shuffle=shuffle)
        edge_index = torch.from_numpy(edge_index).long().to(device)
        labels_list = [torch.from_numpy(item).float().to(device) for item in labels_list]

        # zero grad
        optim.zero_grad()

        # make the predictions
        predictions = model(X, edge_index, use_MSE)[0]

        # Calculating the loss
        if use_MSE:
            predictions = torch.reshape(predictions, (-1,))
            loss = min(criterion(predictions, labels_list[0]), criterion(predictions, labels_list[1]))
        else:
            loss = criterion(predictions, labels_list[0])
            for i in range(1, len(labels_list)):
                loss = min(loss, criterion(predictions, labels_list[i]))

        # backward pass and gradient step
        loss.backward()
        optim.step()

        # Save temporary losses losses and embeddings
        if print_loss:
            losses.append(loss.to('cpu').detach().numpy())

        if print_loss:
            if (graph_num + 1) % checkpoint_save == 0 or (graph_num + 1) == train_graph_num:
                mean_loss = sum(losses) / len(losses)
                mean_losses.append(mean_loss)
                losses = []

        # Print losses
        if print_loss:
            if (graph_num + 1) % checkpoint_print == 0:
                print(f"\tGraph trained: {graph_num+1:,}\t\tLoss: {mean_loss:.4f}")

    if print_loss:
        f = plt.figure()
        f.set_figwidth(10)
        f.set_figheight(5)
        plt.title('Train loss')
        plt.plot(mean_losses, "b")
        plt.show()

    return mean_losses

# make predictions
def result(n, p, q, k, model, train_graph_num, test_graph_num, rand_lab=False,
           istogram=True, use_MSE = False, use_deg=False, shuffle=True, simplier = False):
    accuracys = []
    X = torch.from_numpy(np.eye(n)).float().to(device)


    for graph_num in trange(train_graph_num, train_graph_num + test_graph_num):
        A, edge_index, labels_list = random_graph_generator(n, p, q, k, graph_num, use_MSE=use_MSE, shuffle=shuffle)
        edge_index_test = torch.from_numpy(edge_index).long().to(device)

        if use_deg:
            X = torch.from_numpy(np.diag(np.sum(A, axis=0)) / (np.mean(np.sum(A, axis=0)))).float().to(device)

        # Random label or model
        if rand_lab:
            predictions = np.zeros((n, k))
            interval = n // k
            for i in range(0, k):
                predictions[(0 + interval * i):(interval + interval * i), i] = 1
        else:
            predictions = model(X, edge_index_test, use_MSE)[0].cpu().detach().numpy()

        # initialize results and corrects££
        results = np.zeros(n).astype(int)
        corrects = np.zeros(n)

        # Compare predictions and labels and calculate accuracy
        if use_MSE:
            # correctness of the model
            predictions = np.reshape(predictions, -1)
            accuracy = max(np.mean(np.array(np.sign(predictions) == np.sign(labels_list[0])).astype(int)), np.mean(np.array(np.sign(predictions) == np.sign(labels_list[1])).astype(int)))
            accuracys.append(accuracy * 100)

        else:
            # correctness of the model
            results = [np.argmax(predictions[i,:]) for i in range(n)]
            corrects = [int(np.argmax(labels_list[0][i,:]) == results[i]) for i in range(n)]

            for j in range(1, len(labels_list)):
                corrects_2 = [int(np.argmax(labels_list[j][i,:]) == results[i]) for i in range(n)]
                if np.mean(corrects) * 100 < np.mean(corrects_2) * 100:
                    corrects = corrects_2

            # Accuracy for the Cross Entropy metod
            accuracy = np.mean(corrects) * 100
            accuracys.append(accuracy)

    mean_accuracy = sum(accuracys) / len(accuracys)

    print(f"\nMean accuracy: {mean_accuracy:.2f} %\n\n")

    # Istogram of accuracys
    if istogram:
        f = plt.figure()
        f.set_figwidth(20)
        f.set_figheight(8)
        plt.xlabel('SNR')
        plt.ylabel('Accuracy')
        plt.title('Performance of the model')
        plt.hist(accuracys, bins=20)
        plt.show()

    return accuracys, mean_accuracy

# Spectral cluster
def spectral_clustering(n, q, p, k, train_graph_num, test_graph_num, shuffle=False):
    accuracys = []

    for graph_num in trange(train_graph_num, train_graph_num + test_graph_num):
        A, edge_index, labels_list = random_graph_generator(n, p, q, k, graph_num, shuffle=shuffle)
        D = np.diag(A.sum(axis=0))

        # graph laplacian
        L = D - A

        # eigenvalues and eigenvectors
        vals, vecs = np.linalg.eig(L)

        # sort these based on the eigenvalues
        vecs = vecs[:,np.argsort(vals)]
        vals = vals[np.argsort(vals)]

        # Apply kmeans algorithm
        kmeans = KMeans(n_clusters=k, n_init=15)
        kmeans.fit(vecs[:,1:k])
        results = kmeans.labels_

        # Count the correct label
        corrects = [int(np.argmax(labels_list[0][i,:]) == results[i]) for i in range(n)]
        for j in range(1, len(labels_list)):
            corrects_2 = [int(np.argmax(labels_list[j][i,:]) == results[i]) for i in range(n)]
            if np.mean(corrects) * 100 < np.mean(corrects_2) * 100:
                corrects = corrects_2

        accuracy = np.mean(corrects) * 100
        accuracys.append(accuracy)

    std_spectral_clustering = np.std(accuracys)
    mean_accuracy = sum(accuracys) / len(accuracys)

    print(f"\nMean accuracy: {mean_accuracy:.2f} %\n\n")

    # Istogram of accuracys
    f = plt.figure()
    f.set_figwidth(15)
    f.set_figheight(5)
    plt.title('Accuracy')
    plt.hist(accuracys, bins=20)
    plt.show()

    return mean_accuracy, accuracys, std_spectral_clustering

## Generate a SBM problem

### Initialization of the parameters

In [ ]:
# Initialization
# number of vertices$
n = 300
# probability in clusters
p = 10 / n
# Probability between clusters
q = 5 / n
# Number of cluster
k = 2
# Number of hidden features
h = 1
# Lambda parameter for the regularization (if 0 train without regularization)
par_lambda = 0
# Initialize seed
seed = 2023
# Number of graph to train (thousands)
train_graph_num = 10
# Number of graph to test (thousands)
test_graph_num = 1
# If true use MSE as loss function, if false use cross entropy loss
use_MSE = True
# if 1 use the normal model if 2 use the linear model
model_type = 2



# AUTO INITIALIZATION
# True number of graph
train_graph_num = int(train_graph_num * 1000)
test_graph_num = int(test_graph_num * 1000)

# Total number of graph
tot_graph_num = train_graph_num + test_graph_num

# Use MSE only with 2 clusters
if k != 2:
    use_MSE = False

# Parameters a and b
a = n * p
b = n * q

# check if n is divisible by k, if not change the value of n
if not n % k == 0:
    n = (n // k) * k

# SNR
snr = ((a - b) ** 2)/(k * (a + (k - 1) * b))

# is true shuffle the row and columns of the adjacency matrix (DO NOT CHANGE IT: Experiments without shuffle aren't relevant)
shuffle = True

print(f"Parameter a: {a:.2f}\tParameter b: {b:.2f}\tSNR: {snr:.2f}\tProbability p: {p:.2f}\tProbability q: {q:.2f}\tn: {n}")

### GNN Training

In [ ]:
class GNN(nn.Module):
    def __init__(self, h, k):
        super(GNN, self).__init__()
        torch.manual_seed(seed)

        self.conv_initial = GCNConv(n, h)
        #self.conv_1 = GCNConv(h, h)
        if use_MSE:
            self.conv_final = GCNConv(h, 1, bias=False)
        else:
            self.conv_final = GCNConv(h, k, bias=False)

    def forward(self, X, edge_index, use_MSE):
        preactivations = self.conv_initial(X, edge_index)
        preactivations = F.relu(preactivations)
        X = preactivations
        #X = self.conv_1(X, edge_index)
        #X = F.relu(X)
        X = self.conv_final(X, edge_index)
        if not use_MSE:
            X = F.softmax(X, dim=1)

        return X, preactivations

In [ ]:
class GNN_2(nn.Module):
    def __init__(self, h, k):
        super(GNN_2, self).__init__()
        torch.manual_seed(seed)

        self.conv_initial = GCNConv(n, h, bias=False)
        if use_MSE:
            self.conv_final = nn.Linear(h, 1, bias=False)
        else:
            self.conv_final = GCNConv(h, k, bias=False)

    def forward(self, X, edge_index, use_MSE):
        preactivations = self.conv_initial(X, edge_index)
        X = preactivations
        # X = F.relu(X)
        # X = self.conv_1(X, edge_index)
        # X = F.relu(X)
        X = self.conv_final(X)
        if not use_MSE:
            X = F.softmax(X, dim=1)

        return X, preactivations

In [ ]:
# initialize the model
if model_type == 1:
    model = GNN(h, k).to(device)
else:
    model = GNN_2(h, k).to(device)

In [ ]:
# initialize the loss and the optimizer
if use_MSE:
    criterion = torch.nn.MSELoss().to(device)
else:
    criterion = torch.nn.CrossEntropyLoss().to(device)

optim = torch.optim.Adam(model.parameters(), lr = 5e-2, weight_decay = par_lambda)

In [ ]:
# Layer to freeze ('none', 'initial' or 'final')
name_to_freeze = 'final'

for name, param in model.named_parameters():
    if name_to_freeze in name:
        param.requires_grad = False
        print(f'parameter: {name} freezed')
    else:
        param.requires_grad = True
        print(f'parameter: {name} not freezed')

In [ ]:
# Train the model
train_losses = training_detection(model, optim, criterion, n, p, q, k, train_graph_num, par_lambda, print_loss = True, use_MSE = use_MSE, shuffle=shuffle)

### Results of the training

In [ ]:
# Accuracy and other results
accuracys, mean_accuracy = result(n, p, q, k, model, train_graph_num, test_graph_num, rand_lab=False, istogram=True, use_MSE = use_MSE, shuffle=shuffle)

print(f'n = {n}, p = {p:.2f}, q = {q:.2f}, SNR = {snr:.2f}\n\n')
print(mean_accuracy)
print(np.std(accuracys))

### Spectral clustering

In [ ]:
# Accuracy Spectral clustering
mean_accuracy_spectral_clustering, accuracys_spectral_clustering, std_spectral_clustering = spectral_clustering(n, q, p, k, train_graph_num, test_graph_num, shuffle=shuffle)

### Random assignment


In [ ]:
# Accuracy random assignement
accuracys, mean_accuracy = result(n, p, q, k, model, train_graph_num, test_graph_num, rand_lab=True, shuffle=shuffle)

### Training for different values

In [ ]:
accuracy_random = []
accuracy_spectral = []
accuracy_not_trained = []
accuracy_model = []
SNR = []

random = False
spectral = True
not_trained = False
train_model = False

for a in range(1, 14):
    # Initialization
    # number of vertices$
    n = 500
    # probability in clusters
    p = a / n
    # Probability between clusters
    q = 1 / n
    # Number of cluster
    k = 2
    # Number of hidden features
    h = 50
    # Lambda parameter for the regularization (if 0 train without regularization)
    par_lambda = 0
    # Initialize seed
    seed = 2023
    # Number of graph to train (thousands)
    train_graph_num = 20
    # Number of graph to test (thousands)
    test_graph_num = 1
    # If true use MSE as loss function, if false use cross entropy loss
    use_MSE = True

    # AUTO INITIALIZATION
    # True number of graph
    train_graph_num = int(train_graph_num * 1000)
    test_graph_num = int(test_graph_num * 1000)

    # Total number of graph
    tot_graph_num = train_graph_num + test_graph_num

    # Use MSE only with 2 clusters
    if k != 2:
        use_MSE = False

    # Parameters a and b
    a = n * p
    b = n * q

    # check if n is divisible by k, if not change the value of n
    if not n % k == 0:
        n = (n // k) * k

    # SNR
    snr = ((a - b) ** 2)/(k * (a + (k - 1) * b))
    SNR.append(snr)

    # is true shuffle the row and columns of the adjacency matrix (DO NOT CHANGE IT: Experiments without shuffle aren't relevant)
    shuffle = True

    print(f"Parameter a: {a:.2f}\tParameter b: {b:.2f}\tSNR: {snr:.2f}\tProbability p: {p:.2f}\tProbability q: {q:.2f}\tn: {n}")

    class GNN(nn.Module):
        def __init__(self, h, k):
            super(GNN, self).__init__()
            torch.manual_seed(seed)

            self.conv_initial = GCNConv(n, h)
            #self.conv_1 = GCNConv(h, h)
            if use_MSE:
                self.conv_final = GCNConv(h, 1, bias=False)
            else:
                self.conv_final = GCNConv(h, k, bias=False)

        def forward(self, X, edge_index, use_MSE):
            preactivations = self.conv_initial(X, edge_index)
            preactivations = F.relu(preactivations)
            X = preactivations
            #X = self.conv_1(X, edge_index)
            #X = F.relu(X)
            X = self.conv_final(X, edge_index)
            if not use_MSE:
                X = F.softmax(X, dim=1)

            return X, preactivations

    # initialize the model
    model = GNN(h, k).to(device)

    # initialize the loss and the optimizer
    if use_MSE:
        criterion = torch.nn.MSELoss().to(device)
    else:
        criterion = torch.nn.CrossEntropyLoss().to(device)

    optim = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = par_lambda)

    # Layer to freeze ('none', 'initial' or 'final')
    name_to_freeze = 'none'

    for name, param in model.named_parameters():
        if name_to_freeze in name:
            param.requires_grad = False
            print(f'parameter: {name} freezed')
        else:
            param.requires_grad = True
            print(f'parameter: {name} not freezed')

    # Accuracy not trained
    if not_trained:
        accuracys, mean_accuracy_not_trained = result(n, p, q, k, model, train_graph_num, test_graph_num, rand_lab=False, istogram=False, use_MSE = use_MSE, shuffle=shuffle)
        print(f"Accuracy not trained: {mean_accuracy_not_trained}")
        accuracy_not_trained.append(mean_accuracy_not_trained)

    # Train the model
    if train_model:
        train_losses = training_detection(model, optim, criterion, n, p, q, k, train_graph_num, par_lambda, print_loss = False, use_MSE = use_MSE, shuffle=shuffle)

        # Accuracy model
        accuracys, mean_accuracy_model = result(n, p, q, k, model, train_graph_num, test_graph_num, rand_lab=False, istogram=False, use_MSE = use_MSE, shuffle=shuffle)
        print(f"Accuracy model: {mean_accuracy_model}")
        accuracy_model.append(mean_accuracy_model)

    # Accuracy Spectral clustering
    if spectral:
        mean_accuracy_spectral_clustering, accuracys_spectral_clustering, std_spectral_clustering = spectral_clustering(n, q, p, k, train_graph_num, test_graph_num, shuffle=shuffle)
        print(f"Accuracy spectral clustering: {mean_accuracy_spectral_clustering}")
        accuracy_spectral.append(mean_accuracy_spectral_clustering)

    # Accuracy random assignement
    if random:
        accuracys, mean_accuracy_random = result(n, p, q, k, model, train_graph_num, test_graph_num, rand_lab=True, istogram=False, shuffle=shuffle)
        print(f"Accuracy random: {mean_accuracy_random}")
        accuracy_random.append(mean_accuracy_random)

In [ ]:
# Plot the accuracy for n = 500
accuracy_simple_ones = [51.85339999999996, 52.051999999999936, 52.249600000000044, 52.41960000000003, 52.650999999999996, 52.84460000000002, 53.01439999999998, 53.126200000000075, 53.26839999999993, 53.4268, 53.56980000000002, 53.784600000000054, 53.87540000000007]
SNR_ones = [0.0, 0.023809523809523808, 0.09090909090909091, 0.1956521739130435, 0.3333333333333333, 0.5, 0.6923076923076923, 0.9074074074074074, 1.1428571428571428, 1.396551724137931, 1.6666666666666667, 1.9516129032258065, 2.25, 2.5606060606060606, 2.8823529411764706, 3.2142857142857144, 3.5555555555555554, 3.9054054054054053, 4.2631578947368425, 4.628205128205129]
accuracy_model_MSE = [51.767399999999974, 52.06679999999999, 52.42039999999999, 52.78120000000004, 53.05960000000003, 53.56259999999999, 53.81000000000004, 54.404599999999974, 54.58599999999997, 55.1725999999999, 55.93039999999997, 56.10800000000006, 56.48200000000003]
accuracy_spectral = [50.20559999999943, 50.20759999999948, 50.20779999999949, 50.207199999999446, 50.20759999999943, 50.210599999999445, 50.21399999999945, 50.21559999999942, 50.21239999999943, 50.25559999999948, 50.49459999999945, 50.68979999999948, 51.579599999999424, 53.09699999999944, 56.241199999999324, 60.715399999999256]
accuracy_random = [51.788800000000165, 51.788800000000165, 51.788800000000165, 51.788800000000165, 51.788800000000165, 51.788800000000165, 51.788800000000165, 51.788800000000165, 51.788800000000165, 51.788800000000165, 51.788800000000165, 51.788800000000165, 51.788800000000165]
accuracy_model_CE = [51.74299999999998, 52.19360000000001, 52.541399999999996, 52.844399999999965, 53.19799999999996, 53.41420000000003, 54.024000000000036, 54.652800000000056, 54.70040000000002, 55.465200000000024, 55.74540000000005, 56.50160000000001, 56.92839999999994]
SNR = [5.142857142857143, 4.033333333333333, 3.125, 2.3823529411764706, 1.7777777777777777, 1.2894736842105263, 0.9, 0.5952380952380952, 0.36363636363636365, 0.1956521739130435, 0.08333333333333333, 0.02, 0.0]
accuracy_2_layer_ones = [51.763199999999955, 52.09879999999997, 52.44339999999999, 52.744999999999855, 53.15260000000003, 53.501600000000046, 53.881199999999986, 54.2576, 54.70599999999996, 55.081799999999966, 55.58519999999994, 55.9832, 56.42740000000004]
accuracy_itermediate = [51.902199999999944, 51.98879999999992, 52.22960000000001, 52.38780000000003, 52.6094, 52.76920000000004, 52.93219999999997, 53.067399999999985, 53.251199999999976, 53.406999999999954, 53.5058, 53.676, 53.77439999999994]
SNR_2 = [0.0, 0.16666666666666666, 0.5, 0.9, 1.3333333333333333, 1.7857142857142858, 2.25, 2.7222222222222223, 3.2, 3.6818181818181817, 4.166666666666667, 4.653846153846154, 5.142857142857143]
SNR_spectral = [0.045454545454545456, 0.16666666666666666, 0.34615384615384615, 0.5714285714285714, 0.8333333333333334, 1.125, 1.4411764705882353, 1.7777777777777777, 2.1315789473684212, 2.5, 2.880952380952381, 3.272727272727273, 3.6739130434782608, 4.083333333333333, 4.5, 4.923076923076923]

f_1 = plt.figure()
f_1.set_figwidth(20)
f_1.set_figheight(10)
plt.xlabel('SNR')
plt.ylabel('Mean accuracy')
plt.title('Accuracy with different SNR for 2 communities (20k graph trained)')
plt.plot(SNR_2, accuracy_model_MSE, marker='o', label="Model trained with MSE loss", linewidth=3)
plt.plot(SNR_2, accuracy_model_CE, marker='o', label="Model trained with CE loss", linewidth=3)
plt.plot(SNR_spectral, accuracy_spectral, marker='o', label="Spectral Clustering", linewidth=3)
plt.plot(SNR, accuracy_random, marker='o', label="Random assignement", linewidth=3)
plt.plot(SNR_2, accuracy_simple_ones, marker='o', label="Model Mw", linewidth=3)
plt.plot(SNR_2, accuracy_itermediate, marker='o', label="Model MWθ", linewidth=3)
plt.plot(SNR_2, accuracy_2_layer_ones, marker='o', label="2 layer model f(G)", linewidth=3)
plt.legend(loc="upper left")
f_1.savefig('Accuracy trivial model and intermediate 500 for different SNR.pdf')
plt.show()